In [48]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Step 1: Define paths to your PDF files
pdf_files = ["1.pdf", "2.pdf"]

# Step 2: Load and extract text from PDFs
all_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents = loader.load()
    all_documents.extend(documents)

# Step 3: Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(all_documents)

# Step 4: Initialize HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Lightweight, good for general use

# Step 5: Create FAISS vector store
vector_store = FAISS.from_documents(split_documents, embedding_model)

# Step 6: Save the vector store for future use
vector_store.save_local("cpp_oop_vector_store")

print("Vector store created and saved successfully!")


Vector store created and saved successfully!


In [49]:
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms.cohere import Cohere
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize LLM
cohere_llm = Cohere(cohere_api_key="9rZaWYoS4JsQMcOMZjozd66X0DqbStcRVe8LimVQ", model="command-r-plus")

# Load Vector Store with deserialization flag
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# FAISS vector store loading with `allow_dangerous_deserialization`
vector_store = FAISS.load_local("cpp_oop_vector_store", embedding_model, allow_dangerous_deserialization=True)


In [50]:
# Topics
topics = [
    "Inheritance and Polymorphism in C++",
    "Encapsulation and Abstraction in C++",
    "Virtual Functions and Abstract Classes",
    "STL: Vectors and Iterators",
    "Exception Handling in C++",
    "Templates and Generic Programming",
    "Operator Overloading",
    "File Handling in C++",
    "Multithreading in C++",
    "Design Patterns: Singleton and Factory"
]


# Rubric for evaluation
rubric = """1. Correct use of OOP principles (30 points).
2. Code readability and comments (20 points).
3. Handling of edge cases (20 points).
4. Correctness of solution (30 points)."""

# Function to retrieve context from vector store
def retrieve_relevant_info(topic):
    """Retrieve the most relevant information for the topic from the vector store."""
    results = vector_store.similarity_search(topic, k=3)
    return "\n\n".join([result.page_content for result in results])

def generate_problem(topic):
    """
    Generates a detailed C++ programming problem in the specified OOP topic, enriched with relevant context.
    """
    # Retrieve related information
    relevant_info = retrieve_relevant_info(topic)

    # Create the problem prompt enriched with retrieved info
    problem_prompt = PromptTemplate(
        input_variables=["topic", "relevant_info"],
        template=(
            "You are a C++ programming instructor. Create a detailed programming problem in the area of {topic}. "
            "Use the following information to help design the problem:\n{relevant_info}\n"
            "Include a clear rubric, deliverables, and requirements. Make the problem challenging yet solvable for students."
        )
    )

    # Set up the problem generation chain using the defined cohere_llm
    problem_chain = LLMChain(llm=cohere_llm, prompt=problem_prompt)  # Use cohere_llm here
    return problem_chain.run(topic=topic, relevant_info=relevant_info)


def evaluate_code(problem, rubric, student_code):
    """
    Evaluates the student's C++ code against the problem statement and rubric, enriched with additional context.
    """
    # Retrieve common mistakes related to the topic for context
    common_mistakes = retrieve_relevant_info("common mistakes in OOP programming")

    # Create the evaluation prompt enriched with retrieved info
    evaluation_prompt = PromptTemplate(
        input_variables=["problem", "rubric", "student_code", "common_mistakes"],
        template=(
            "You are a C++ programming evaluator. The problem statement is as follows:\n{problem}\n\n"
            "Rubric for evaluation:\n{rubric}\n\n"
            "The student's code submission is:\n{student_code}\n\n"
            "Common mistakes for this topic include:\n{common_mistakes}\n\n"
            "Provide a detailed evaluation based on the rubric, analyzing the correctness, design, and any improvements needed."
        )
    )

    # Set up the evaluation chain using the defined cohere_llm
    evaluation_chain = LLMChain(llm=cohere_llm, prompt=evaluation_prompt)  # Use cohere_llm here
    return evaluation_chain.run(problem=problem, rubric=rubric, student_code=student_code, common_mistakes=common_mistakes)


# Generate Feedback
def generate_feedback(evaluation):
    """Generates constructive feedback based on the evaluation."""
    feedback_tips = retrieve_relevant_info("feedback tips for OOP programming")

    feedback_prompt = PromptTemplate(
        input_variables=["evaluation", "feedback_tips"],
        template=(
            "You are an instructor. Based on the following evaluation, provide detailed feedback for the student:\n"
            "{evaluation}\n\n"
            "Use these tips to guide the feedback:\n{feedback_tips}\n\n"
            "Focus on strengths, weaknesses, and steps for improvement."
        ),
    )

    feedback_chain = LLMChain(llm=cohere_llm, prompt=feedback_prompt)
    return feedback_chain.run(evaluation=evaluation, feedback_tips=feedback_tips)

# Main Workflow
def run_rag_pipeline(topics, student_solutions):
    """
    Full RAG workflow: generates problems, evaluates student solutions, and provides feedback.

    Args:
        topics: List of topics to generate problems for.
        student_solutions: List of corresponding student code solutions.
    """
    assert len(topics) == len(student_solutions), "Number of topics must match number of solutions!"

    # Output file
    output_file = "cpp_oop_rag_results.txt"

    with open(output_file, "w") as file:
        for i, topic in enumerate(topics):
            file.write(f"_________________topic number {i}___________________\n\n")
            # Step 1: Generate Problem
            problem = generate_problem(topic)
            file.write(f"Problem for Topic {i+1} ({topic}):\n{problem}\n\n")


            # Step 2: Evaluate Student Solution
            student_code = student_solutions[i]
            #Step 1.9999: Write student_solutions

            file.write(f"Student Solution for Topic {i+1}:\n{student_code}\n\n")
            evaluation = evaluate_code(problem, rubric, student_code)
            file.write(f"Evaluation for Topic {i+1}:\n{evaluation}\n\n")

            # Step 3: Generate Feedback
            feedback = generate_feedback(evaluation)
            file.write(f"Feedback for Topic {i+1}:\n{feedback}\n\n")


    print(f"Results have been written to {output_file}")

# Example usage
if __name__ == "__main__":
    # Placeholder student solutions for 10 topics
    # Sample topics for testing
    sample_topics = [
        "Inheritance and Polymorphism in C++",
        "Encapsulation and Abstraction in C++",
        "Virtual Functions and Abstract Classes",
        "STL: Vectors and Iterators",
        "Exception Handling in C++",
        "Templates and Generic Programming",
        "Operator Overloading",
        "File Handling in C++",
        "Multithreading in C++",
        "Design Patterns: Singleton and Factory"
    ]


    sample_solutions = [
        # Solution 1: Inheritance and Polymorphism in C++
        """
        #include <iostream>
        using namespace std;

        class Animal {
        public:
            virtual void sound() {
                cout << "Animal makes a sound" << endl;
            }
        };

        class Dog : public Animal {
        public:
            void sound() override {
                cout << "Dog barks" << endl;
            }
        };

        int main() {
            Animal* obj = new Dog();
            obj->sound();
            delete obj;
            return 0;
        }
        """,
        # Solution 2: Encapsulation and Abstraction in C++
        """
        #include <iostream>
        using namespace std;

        class BankAccount {
        private:
            double balance;

        public:
            BankAccount(double initialBalance) : balance(initialBalance) {}

            void deposit(double amount) {
                if (amount > 0) {
                    balance += amount;
                }
            }

            double getBalance() const {
                return balance;
            }
        };

        int main() {
            BankAccount account(1000);
            account.deposit(500);
            cout << "Balance: " << account.getBalance() << endl;
            return 0;
        }
        """,
        # Solution 3: Virtual Functions and Abstract Classes
        """
        #include <iostream>
        using namespace std;

        class Shape {
        public:
            virtual double area() const = 0; // Pure virtual function
            virtual void display() const = 0;
        };

        class Circle : public Shape {
        private:
            double radius;

        public:
            Circle(double r) : radius(r) {}

            double area() const override {
                return 3.14159 * radius * radius;
            }

            void display() const override {
                cout << "Circle with area: " << area() << endl;
            }
        };

        int main() {
            Shape* shape = new Circle(5);
            shape->display();
            delete shape;
            return 0;
        }
        """,
        # Solution 4: STL - Vectors and Iterators
        """
        #include <iostream>
        #include <vector>
        using namespace std;

        int main() {
            vector<int> numbers = {1, 2, 3, 4, 5};

            for (auto it = numbers.begin(); it != numbers.end(); ++it) {
                cout << *it << " ";
            }
            cout << endl;

            return 0;
        }
        """,
        # Solution 5: Exception Handling in C++
        """
        #include <iostream>
        using namespace std;

        int main() {
            try {
                int a = 10, b = 0;
                if (b == 0) {
                    throw runtime_error("Division by zero!");
                }
                cout << "Result: " << a / b << endl;
            } catch (const exception& e) {
                cout << "Error: " << e.what() << endl;
            }
            return 0;
        }
        """,
        # Solution 6: Templates and Generic Programming
        """
        #include <iostream>
        using namespace std;

        template <typename T>
        T add(T a, T b) {
            return a + b;
        }

        int main() {
            cout << "Sum of integers: " << add<int>(5, 10) << endl;
            cout << "Sum of doubles: " << add<double>(3.5, 7.8) << endl;
            return 0;
        }
        """,
        # Solution 7: Operator Overloading
        """
        #include <iostream>
        using namespace std;

        class Complex {
        private:
            double real, imag;

        public:
            Complex(double r, double i) : real(r), imag(i) {}

            Complex operator+(const Complex& other) const {
                return Complex(real + other.real, imag + other.imag);
            }

            void display() const {
                cout << real << " + " << imag << "i" << endl;
            }
        };

        int main() {
            Complex c1(3, 4), c2(1, 2);
            Complex c3 = c1 + c2;
            c3.display();
            return 0;
        }
        """,
        # Solution 8: File Handling in C++
        """
        #include <iostream>
        #include <fstream>
        using namespace std;

        int main() {
            ofstream outFile("example.txt");
            outFile << "Hello, file!" << endl;
            outFile.close();

            ifstream inFile("example.txt");
            string content;
            getline(inFile, content);
            cout << "Read from file: " << content << endl;
            inFile.close();

            return 0;
        }
        """,
        # Solution 9: Multithreading in C++
        """
        #include <iostream>
        #include <thread>
        using namespace std;

        void printNumbers() {
            for (int i = 1; i <= 5; ++i) {
                cout << i << " ";
            }
            cout << endl;
        }

        int main() {
            thread t1(printNumbers);
            thread t2(printNumbers);

            t1.join();
            t2.join();

            return 0;
        }
        """,
        # Solution 10: Singleton Design Pattern
        """
        #include <iostream>
        using namespace std;

        class Singleton {
        private:
            static Singleton* instance;
            Singleton() {}

        public:
            static Singleton* getInstance() {
                if (!instance) {
                    instance = new Singleton();
                }
                return instance;
            }
        };

        Singleton* Singleton::instance = nullptr;

        int main() {
            Singleton* s1 = Singleton::getInstance();
            Singleton* s2 = Singleton::getInstance();
            cout << (s1 == s2) << endl; // Output: 1 (true)
            return 0;
        }
        """
    ]
    run_rag_pipeline(sample_topics, sample_solutions)


Results have been written to cpp_oop_rag_results.txt
Results have been written to cpp_oop_rag_results.txt
